In [56]:
import torch
from pathlib import Path

In [57]:
cpt_fp = Path("shakespeare-char/nano-h4/ckpt.pt")
cpt = torch.load(cpt_fp)

In [58]:
def gen_orth():
    orth_linear = torch.nn.utils.parametrizations.orthogonal(torch.nn.Linear(32, 32))
    return orth_linear.weight
Q = gen_orth()
with torch.no_grad():
    print(torch.dist(Q.T @ Q, torch.eye(32)))

tensor(2.0165e-06)


In [59]:
for l, v in cpt["model"].items():
    if not "c_attn" in l:
        continue
    heads = v.split(128 // 4, dim=0)
    orths = {}
    new_heads = []
    for i, h in enumerate(heads):
        if i >= 8:
            new_heads.append(h)
            continue
        im = i % 4
        if im in orths:
            Q = orths[im]
        else:
            Q = gen_orth()
            orths[im] = Q
        new_heads.append(Q @ h)
    new_matrix = torch.cat(new_heads, dim=0)
    print(new_matrix.size())
    cpt["model"][l] = new_matrix

torch.Size([384, 128])
torch.Size([384, 128])
torch.Size([384, 128])
torch.Size([384, 128])


In [60]:
new_path = Path(
    "shakespeare-char-rotated", cpt_fp.parent.name, "ckpt.pt"
)
new_path.parent.mkdir(exist_ok=True, parents=True)
torch.save(cpt, new_path)

In [64]:
cpt1 = torch.load(cpt_fp)
cpt2 = torch.load(new_path)

for k, v in cpt1["model"].items():
    with torch.no_grad():
        delta = torch.abs(v - cpt2["model"][k]).sum()
    print(f"{k:<50}{delta.item():.2f}")

transformer.wte.weight                            0.00
transformer.wpe.weight                            0.00
transformer.h.0.ln_1.weight                       0.00
transformer.h.0.attn.c_attn.weight                2090.09
transformer.h.0.attn.c_proj.weight                0.00
transformer.h.0.ln_2.weight                       0.00
transformer.h.0.mlp.c_fc.weight                   0.00
transformer.h.0.mlp.c_proj.weight                 0.00
transformer.h.1.ln_1.weight                       0.00
transformer.h.1.attn.c_attn.weight                2033.05
transformer.h.1.attn.c_proj.weight                0.00
transformer.h.1.ln_2.weight                       0.00
transformer.h.1.mlp.c_fc.weight                   0.00
transformer.h.1.mlp.c_proj.weight                 0.00
transformer.h.2.ln_1.weight                       0.00
transformer.h.2.attn.c_attn.weight                2388.94
transformer.h.2.attn.c_proj.weight                0.00
transformer.h.2.ln_2.weight                       0.00
t